In [2]:
#Library imports
import re
import os
import sys
import shutil
import time
import math
import gzip
import fnmatch
import random
import warnings
import numpy as np
import pandas as pd
import scipy.stats as scs
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt

from pandas.tseries.offsets import DateOffset
from collections import OrderedDict

import scipy.stats as scs
from sklearn.neighbors import BallTree, KDTree

import fiona
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from pyproj import Proj
import geoplot as gplt
import geoplot.crs as gcrs

# Allows access to scripts and modules relative to the parent directory.
parent = os.getcwd()
sys.path.append(os.path.join(parent, "../functions"))

# Project specific user driven functions
from cleaning_functions import *

# My open source reusable user driven function repository.
from random_lumberjacks.src.random_lumberjacks.cleaning.cleaning_functions import *
from random_lumberjacks.src.random_lumberjacks.model.model_classes import *
from random_lumberjacks.src.random_lumberjacks.visualization.visualization_functions import *
from random_lumberjacks.src.random_lumberjacks.parsing.parse_noaa import *

#Notebook arguments
%matplotlib inline

In [2]:
station_key = pd.read_csv("data/noaa/isd-history.csv")

In [3]:
all_states = station_key[(station_key["CTRY"]=="US")]["STATE"].dropna().unique()
airshed = ['DE','IN','KY','MD','MI','MI','NC','NJ','NY','OH','PA','SC','TN','VA','VT','WV']
not_airshed = np.setdiff1d(all_states, airshed)

In [4]:
#Converting the end date to datetime to be able to select relevant years.
station_key["END"] = pd.to_datetime(station_key["END"], format="%Y%m%d")

#All data must have coordinates, be within the US, not be explicitly outside of the airshed, and be from 2001 or later.
relevant_stations = station_key[(station_key["CTRY"]=="US") & (station_key["STATE"].isin(not_airshed) == False) &
                                (station_key["END"] > "2001") & (station_key["LAT"].isna()==False)].reset_index().drop(columns = "index")


In [5]:
relevant_stations

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,621010,99999,MOORED BUOY,US,NaN,NaN,50.600,-2.933,-999.0,20080721,2008-07-21
1,621110,99999,MOORED BUOY,US,NaN,NaN,58.900,-0.200,-999.0,20041118,2004-11-18
2,621130,99999,MOORED BUOY,US,NaN,NaN,58.400,0.300,-999.0,20040726,2004-07-26
3,621160,99999,MOORED BUOY,US,NaN,NaN,58.100,1.800,-999.0,20040829,2004-08-29
4,621170,99999,MOORED BUOY,US,NaN,NaN,57.900,0.100,-999.0,20040726,2004-07-26
...,...,...,...,...,...,...,...,...,...,...,...
1533,A06773,334,TUCKER GUTHRIE MEMORIAL AIRPORT,US,KY,KI35,36.859,-83.358,473.1,20140731,2020-08-24
1534,A06800,120,TAZEWELL COUNTY AIRPORT,US,VA,KJFZ,37.067,-81.800,808.6,20140731,2020-08-24
1535,A06884,416,LURAY CAVERNS AIRPORT,US,VA,KLUA,38.667,-78.501,275.2,20140731,2020-08-24
1536,A07086,468,CARL R KELLER FIELD AIRPORT,US,OH,KPCW,41.516,-82.869,179.8,20140731,2020-08-24


In [6]:
cbp_cmc = pd.read_pickle("data/cbp_cmc.pickle")

In [7]:
locations_a = cbp_cmc.groupby(["Station"]).first().reset_index()[["Station", "Latitude", "Longitude"]]
locations_b = relevant_stations[["USAF", "LAT", "LON"]]
location_key = locations_a.copy()

In [8]:
tree = BallTree(np.deg2rad(locations_b[["LAT", "LON"]].values), metric='haversine')

In [9]:
distances, indices = tree.query(np.deg2rad(locations_a[["Latitude", "Longitude"]]), k = 3)
indices = pd.DataFrame(indices, columns = [f"id{i}" for i in np.arange(1,4)])
distances = pd.DataFrame(distances*3959, columns = [f"noaa_dist_mi{i}" for i in np.arange(1,4)])

In [10]:
for i, column in enumerate(indices.columns):
    location_key["USAF_"+column] = indices[column].map(lambda x: relevant_stations["USAF"][x])
    location_key["WBAN_"+column] = indices[column].map(lambda x: relevant_stations["WBAN"][x])
    location_key[f"noaa_dist_mi{i+1}"] = distances[f"noaa_dist_mi{i+1}"]

In [23]:
def raw_noaa_to_dataframe(data, fixed_locs, optional_locs=None):
    columns = [item[0] for item in [*fixed_locs]]
    #Adds optional column names if they exist.
    if optional_locs:
        for block in optional_locs:
            columns.extend([item[0] for item in optional_locs[block][1]])
    noaa_list = []
    for line in data:
        var = line[108:].split(b" ")
        fixed_data = parse_fixed_noaa_data(line, fixed_locs)
        optional_data = parse_optional_noaa_data(line, optional_locs)
        data = fixed_data + optional_data
        noaa_list.append(data)
    df = pd.DataFrame(noaa_list, columns = columns, dtype=object)
    fix_noaa_df_dtypes(df, fixed_locs, optional_locs)
    return df

In [24]:
key = ["column_name", "start", "end", "dtype", "nan_value", "conversion factor"]

fixed_locs = [["USAF_ID", 4, 10, "str", None, None], ["NCEI_WBAN_ID", 10, 15, "str", None, None],
              ["Date", 15, 27, "datetime", None, None], ["Data Source", 27, 28, "str", "9", None],
              ["Latitude", 28, 34, "float64", "+99999", 1000], ["Longitude", 34, 41, "float64", "+999999", 1000],
              ["Code", 41, 46, "str", "99999", None], ["Elevation", 46, 51, "float64", "+9999", None],
              ["Call_Letter", 51, 56, "str", "99999", None], ["Quality_Control", 56, 60, "str", "99999", None],
              ["Wind_Dir", 60, 63, "float64", "999", None], ["Wind_Dir_Q", 63, 64, "str", None, None],
              ["Wind_Type", 64, 65, "str", "9", None], ["Wind_Speed", 65, 69, "float64", "9999", 10],
              ["Wind_Speed_Q", 69, 70, "str", None, None], ["Air Temperature", 87, 92, "float64", "+9999", 10],
              ["Air Temperature_Q", 92, 93, "str", None, None], ["Air_Pressure", 99, 104, "float64", "99999", 10],
              ["Air_Pressure_Q", 104, 105, "str", None, None]]

optional_locs = OrderedDict([("AA1",[11, [["Rain_Period1", 3, 5, "str", "99", None],
                                          ["Rain_Depth1", 5, 9, "str", "9999", 10],
                                          ["Rain_Condition1", 9, 10, "str", "9", None],
                                          ["Rain_Qual1", 10, 11, "str", None, None]]]),
                             ("AA2",[11, [["Rain_Period2", 3, 5, "str", "99", None],
                                          ["Rain_Depth2", 5, 9, "str", "9999", 10],
                                          ["Rain_Condition2", 9, 10, "str", "9", None],
                                          ["Rain_Qual2", 10, 11, "str", None, None]]]),
                             ("AA3",[11, [["Rain_Period3", 3, 5, "str", "99", None],
                                          ["Rain_Depth3", 5, 9, "str", "9999", 10],
                                          ["Rain_Condition3", 9, 10, "str", "9", None],
                                          ["Rain_Qual3", 10, 11, "str", None, None]]]),
                             ("AA4",[11, [["Rain_Period4", 3, 5, "str", "99", None],
                                          ["Rain_Depth4", 5, 9, "str", "9999", 10],
                                          ["Rain_Condition4", 9, 10, "str", "9", None],
                                          ["Rain_Qual4", 10, 11, "str", None, None]]])])


In [31]:
dir(station["Date"].dt)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_add_delegate_accessors',
 '_constructor',
 '_delegate_method',
 '_delegate_property_get',
 '_delegate_property_set',
 '_deprecations',
 '_dir_additions',
 '_dir_deletions',
 '_freeze',
 '_get_values',
 '_reset_cache',
 'ceil',
 'date',
 'day',
 'day_name',
 'dayofweek',
 'dayofyear',
 'days_in_month',
 'daysinmonth',
 'floor',
 'freq',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_month_start',
 'is_quarter_end',
 'is_quarter_start',
 'is_year_end',
 'is_year_start',
 'isocalendar',
 'microsecond',
 'minute',
 'month',
 'month_name',
 'nanosecond',
 'normalize',
 'quarter',
 'round',

In [78]:
    # If no data for rain neighboring hours will be used instead to reduce observations.
    hours = station_filter.set_index("Date").groupby(pd.Grouper(freq='H')).count().index
    
    for i in range(-3,4):
        hours = hours.union(hours+DateOffset(hours=i))

,Station,Agency,CloudCover,Cruise,Database,FieldActivityEventType,FlowStage,GaugeHeight,GroupCode,HUC12,...,Weather Conditions Yesterday,WindDirection,WindSpeed,Point,HUC12_,HUCNAME_,FIPS_,WATER TEMPERATURE DEG,SPECIFIC CONDUCTIVITY,AIR TEMPERATURE DEG
Date,,,,,,,,,,,,,,,,,,,,,
2005-07-11 09:00:00,DRN010.53,VADEQ,Overcast (>90%),BAY422,CBP,NaN,NaN,NaN,NaN,2.080102e+10,...,NaN,SW,0 To 1 Knot,POINT (-76.6964 37.63422),2.080102e+10,Meggs Bay-Dragon Swamp,51097.0,23.16,NaN,NaN
2005-07-11 10:00:00,PNK013.91,VADEQ,Partly To Broken (50-90%),BAY422,CBP,NaN,NaN,NaN,NaN,2.080102e+10,...,NaN,SSW,0 To 1 Knot,POINT (-76.48782 37.53407),2.080102e+10,Carvers Creek-Piankatank River,51073.0,28.87,NaN,NaN
2005-07-11 11:00:00,PNK018.35,VADEQ,Overcast (>90%),BAY422,CBP,NaN,NaN,NaN,NaN,2.080102e+10,...,NaN,SW,1 To 10 Knots,POINT (-76.54648 37.56677),2.080102e+10,Carvers Creek-Piankatank River,51073.0,25.97,NaN,NaN
2005-07-13 09:00:00,TSK000.23,VADEQ,Partly To Broken (50-90%),BAY422,CBP,NaN,NaN,NaN,NaN,2.080107e+10,...,NaN,W,0 To 1 Knot,POINT (-76.71441 37.41497),2.080107e+10,Skimino Creek-York River,51095.0,29.35,NaN,NaN
2005-07-14 10:00:00,PMK012.18,VADEQ,Hazy,BAY422,CBP,NaN,NaN,NaN,NaN,2.080106e+10,...,NaN,W,0 To 1 Knot,POINT (-76.88424000000001 37.57138),2.080106e+10,Mill Creek-Pamunkey River,51101.0,28.73,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-24 10:00:00,7-DRN010.48,VADEQ,NaN,NTN020,CBP,R,NaN,5.18,NaN,2.080102e+10,...,NaN,NaN,NaN,POINT (-76.69583 37.63361),2.080102e+10,Meggs Bay-Dragon Swamp,51119.0,10.02,NaN,NaN
2020-03-30 15:00:00,ACB.BLACRE0.6,NaN,NaN,NaN,CMC,NaN,NaN,NaN,ACB,NaN,...,NaN,NaN,NaN,POINT (-76.64814920000001 37.4070413),2.080107e+10,Skimino Creek-York River,51073.0,19.60,NaN,27.1
2020-04-29 16:00:00,ACB.BLACRE0.6,NaN,NaN,NaN,CMC,NaN,NaN,NaN,MPMN,NaN,...,NaN,NaN,NaN,POINT (-76.64814920000001 37.4070413),2.080107e+10,Skimino Creek-York River,51073.0,20.40,NaN,24.8


In [69]:
dates = station_filter.set_index("Date").groupby(pd.Grouper(freq='D')).count().index

In [50]:
hours[(hours>="2015") & (hours <"2016")]

DatetimeIndex(['2015-01-01 00:00:00', '2015-01-01 01:00:00',
               '2015-01-01 02:00:00', '2015-01-01 03:00:00',
               '2015-01-01 04:00:00', '2015-01-01 05:00:00',
               '2015-01-01 06:00:00', '2015-01-01 07:00:00',
               '2015-01-01 08:00:00', '2015-01-01 09:00:00',
               ...
               '2015-12-31 14:00:00', '2015-12-31 15:00:00',
               '2015-12-31 16:00:00', '2015-12-31 17:00:00',
               '2015-12-31 18:00:00', '2015-12-31 19:00:00',
               '2015-12-31 20:00:00', '2015-12-31 21:00:00',
               '2015-12-31 22:00:00', '2015-12-31 23:00:00'],
              dtype='datetime64[ns]', name='Date', length=8760, freq='H')

In [46]:
station["Date"]

0       2015-01-01 00:15:00
1       2015-01-01 00:35:00
2       2015-01-01 00:55:00
3       2015-01-01 01:15:00
4       2015-01-01 01:35:00
                ...        
25932   2015-12-31 22:35:00
25933   2015-12-31 22:55:00
25934   2015-12-31 23:15:00
25935   2015-12-31 23:35:00
25936   2015-12-31 23:55:00
Name: Date, Length: 25937, dtype: datetime64[ns]

In [43]:
station["Date"].round("H").isin(pd.Series(hours))

0        True
1        True
2        True
3        True
4        True
         ... 
25932    True
25933    True
25934    True
25935    True
25936    True
Name: Date, Length: 25937, dtype: bool

In [36]:
station[(station["Date"].dt.date.isin(pd.Series(hours).dt.date))]

,USAF_ID,NCEI_WBAN_ID,Date,Data Source,Latitude,Longitude,Code,Elevation,Call_Letter,Quality_Control,...,Rain_Condition2,Rain_Qual2,Rain_Period3,Rain_Depth3,Rain_Condition3,Rain_Qual3,Rain_Period4,Rain_Depth4,Rain_Condition4,Rain_Qual4
0,720285,03734,2015-01-01 00:15:00,4,37.521,-76.765,FM-15,7.0,NaN,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,720285,03734,2015-01-01 00:35:00,4,37.521,-76.765,FM-15,7.0,NaN,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,720285,03734,2015-01-01 00:55:00,4,37.521,-76.765,FM-15,7.0,NaN,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,720285,03734,2015-01-01 01:15:00,4,37.521,-76.765,FM-15,7.0,NaN,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,720285,03734,2015-01-01 01:35:00,4,37.521,-76.765,FM-15,7.0,NaN,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25932,720285,03734,2015-12-31 22:35:00,4,37.521,-76.765,FM-15,7.0,NaN,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25933,720285,03734,2015-12-31 22:55:00,4,37.521,-76.765,FM-15,7.0,NaN,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25934,720285,03734,2015-12-31 23:15:00,4,37.521,-76.765,FM-15,7.0,NaN,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25935,720285,03734,2015-12-31 23:35:00,4,37.521,-76.765,FM-15,7.0,NaN,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_dfs = pd.DataFrame()
stations_to_query = pd.Index([])
for i in np.arange(1,4):
    group = location_key[f"USAF_id{i}"] + "-" + location_key[f"WBAN_id{i}"].map(lambda x: f"{x:05d}")
    stations_to_query = stations_to_query.union(group).unique()

station_filepaths = select_noaa_files(stations_to_query, "data/noaa", 2001, 2020)

for station_query in stations_to_query:
    station_df = pd.DataFrame()
    
    cut = station_query.split("-")
    mask = pd.Series(False, index=location_key.index)
    for i in np.arange(1,4):
        partial_mask = (location_key[f"USAF_id{i}"] == cut[0]) & (location_key[f"WBAN_id{i}"] == int(cut[1]))
        mask = mask | partial_mask
    station_matches = location_key[mask]
    
    # Creates an index of dates for the current station to restrict the size of the Data imported from NOAA
    station_filter = cbp_cmc[cbp_cmc["Station"].isin(station_matches["Station"])]
    dates = station_filter.set_index("Date").groupby(pd.Grouper(freq='D')).first().dropna(how="all").index

    # Allows for collection of stats for previous two days.
    for i in np.arange(1,3):
        dates = dates.union(dates-DateOffset(days=i))
        

    
    reduced_filepaths = station_filepaths[station_filepaths.str.contains(station_query)]
    for filepath in reduced_filepaths:
        print(filepath)
        raw_data = noaa_gzip_to_raw(filepath)
        station = raw_noaa_to_dataframe(raw_data, fixed_locs, optional_locs)
        #Trims out unneccary dates
        station = station[station["Date"].dt.date.isin(pd.Series(dates).dt.date)]
        station_df = pd.concat([station_df, station])
    all_dfs = pd.concat([all_dfs, station_df])

In [135]:
df = all_dfs.drop(columns = ["Rain_Period4", "Rain_Depth4", "Rain_Condition4", "Rain_Qual4"]).reset_index().drop(columns="index").copy()

In [136]:
df["Rain_Depth2"] = np.where(df["Rain_Depth2"]== '3;', np.nan, df["Rain_Depth2"])
df["Rain_Depth3"] = np.where(df["Rain_Depth3"]== '7;', np.nan, df["Rain_Depth3"])
for i in np.arange(1,4):
    df[f"Rain_Depth{i}"] = df[f"Rain_Depth{i}"].astype("float64")/10

In [137]:
df

,USAF_ID,NCEI_WBAN_ID,Date,Data Source,Latitude,Longitude,Code,Elevation,Call_Letter,Quality_Control,...,Rain_Condition1,Rain_Qual1,Rain_Period2,Rain_Depth2,Rain_Condition2,Rain_Qual2,Rain_Period3,Rain_Depth3,Rain_Condition3,Rain_Qual3
0,720285,03734,2006-01-01 00:03:00,4,37.517,-76.767,AUTO,7.0,KFYJ,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,720285,03734,2006-01-01 00:19:00,4,37.517,-76.767,AUTO,7.0,KFYJ,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,720285,03734,2006-01-01 00:39:00,4,37.517,-76.767,AUTO,7.0,KFYJ,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,720285,03734,2006-01-01 00:59:00,7,37.521,-76.765,FM-15,7.0,KFYJ,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,720285,03734,2006-01-01 01:23:00,4,37.517,-76.767,AUTO,7.0,KFYJ,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10097286,A06884,00416,2020-03-19 22:35:00,6,38.667,-78.501,FM-15,275.0,KLUA,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10097287,A06884,00416,2020-03-19 22:55:00,6,38.667,-78.501,FM-15,275.0,KLUA,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10097288,A06884,00416,2020-03-19 23:15:00,6,38.667,-78.501,FM-15,275.0,KLUA,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10097289,A06884,00416,2020-03-19 23:35:00,6,38.667,-78.501,FM-15,275.0,KLUA,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
df.to_pickle("data/noaa_match.pickle")
location_key.to_pickle("data/noaa_location_key.pickle")